## Red Neuronal Artificial (ANN)

Red neuronal artificial (ANN) avanzada para regresión utilizando TensorFlow y Keras. He incorporado las mejores prácticas, como la normalización de datos, regularización para evitar el sobreajuste, early stopping y ajuste de hiperparámetros.

La red está diseñada para manejar datos de regresión complejos, por lo que incluye varias capas densas con activaciones ReLU, dropout y regularización L2.

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
# Generar datos sintéticos para regresión
np.random.seed(42)
X = np.random.rand(5000, 10)  # 5000 muestras, 10 características
y = np.sum(X * np.random.rand(10), axis=1) + np.random.normal(0, 0.1, 5000)  # Función objetivo con ruido

# Dividir en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
# Normalización de los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
# Definir la arquitectura de la red neuronal
def build_model():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(X_train.shape[1],)),
        layers.Dropout(0.2),  # Dropout para evitar sobreajuste
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dense(1)  # Capa de salida sin activación para regresión
    ])

    # Compilar el modelo con optimizador Adam y métrica de error cuadrático medio (MSE)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  loss='mse',
                  metrics=['mae'])

    return model

In [ ]:
# Construcción del modelo
model = build_model()

# Configurar el callback de Early Stopping para evitar sobreajuste
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [ ]:
# Entrenamiento del modelo
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_val, y_val),
                    callbacks=[early_stopping], verbose=1)

In [ ]:
# Evaluación en el conjunto de prueba
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"\nPérdida en el conjunto de prueba: {test_loss:.4f}")
print(f"MAE en el conjunto de prueba: {test_mae:.4f}")

# Gráfico de la pérdida durante el entrenamiento
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida (MSE)')
plt.legend()
plt.title('Evolución de la pérdida')
plt.show()

## Explicación de la implementación:

1. Preprocesamiento de datos:
- Se generan datos sintéticos con 10 características.
- Se divide el dataset en entrenamiento, validación y prueba.
- Se normalizan las características con StandardScaler.

2. Definición del modelo:
- Capas densas con activación ReLU para manejar no linealidades.
- Regularización L2 (kernel_regularizer) para evitar sobreajuste.
- Dropout en capas ocultas para mejorar la generalización.

3. Compilación y entrenamiento:
- Optimización con Adam (learning_rate=0.001).
- Función de pérdida MSE (error cuadrático medio), adecuada para regresión.
- Callback de EarlyStopping para detener el entrenamiento cuando la pérdida de validación deja de mejorar.

4. Evaluación y visualización:
- Se mide el error en el conjunto de prueba.
- Se grafica la evolución de la pérdida en entrenamiento y validación.